# Fourierův zákon

Podle Fourierova zákona je možné získat tok tepla jako součin součinitele tepelné vodivosti materiálu a spádu teploty (záporně vzatého gradientu teploty). Pro anizotropní materiály je však součinitel tepelné vodivosti maticový, což s sebou nese několik komplikací.

* Při stejném poklesu teploty na jednotku délky nemusí být stejný tepelný tok.
* Tepelný tok nemusí být rovnoběžný se směrem poklesu teploty.

Níže si toto ilustrujeme na příkladě dvou materiálů: izotropního hliníku a anizotropního dřeva.

In [3]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=4)
aluminium_conductivity = 237 # W / (m * K)  
wood_conductivity = np.array([0.3, 0, 0, 0.180]).reshape(2,2)   # https://www.dashofer.cz/download/pdf/drv/2_5_1_tepelne_vlastnosti_dreva.pdf , javor


In [13]:
def test_parallel(a,b):
    """
    Returns True if vectors a and b are parallel
    """
    return np.abs((np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)))-1)<0.0001

data = {}
np.random.seed(42)
v = np.random.rand(2)
v = v/np.linalg.norm(v)
spady_teploty = [(1,0),(0,1),(1,1)*1/np.sqrt(2), v]
dec_places = 5
for i,spad_teploty in enumerate(spady_teploty):
    # vypocet toku
    spad_teploty = np.array(spad_teploty)
    tok_al = aluminium_conductivity * spad_teploty  # tok tepla v hliniku
    tok_wood = wood_conductivity @ spad_teploty  # tok tepla ve dreve, pouzivame maticove nasobeni @
    # zaokrouhleni
    # spad_teploty, tok_al, tok_wood = list(map(lambda x:np.around(x,dec_places),[spad_teploty, tok_al, tok_wood]))
    # ulozeni do tabulky
    data[(i,'spád teploty')] = [spad_teploty] * 2
    data[(i,'velikost spádu teploty')] = [np.linalg.norm(spad_teploty)]*2
    toky = [tok_al, tok_wood]
    data[(i,'tok')] = toky
    data[(i,'velikost toku')] = [np.linalg.norm(x) for x in toky]
    data[(i,'test rovnoběžnosti')] = [test_parallel(x,spad_teploty) for x in toky]

df = pd.DataFrame(data).T
df.columns = ["Hliník", "Dřevo"]
polozky = np.unique([i[1] for i in df.index])
df.style.format(precision=3)

# Čeho si všímat

* Jsou zadány čtyři možnosti spádu teploty. Tyto možnosti se liší pouze směrem, velikost spádu je vždy jeden Kelvin na metr. První dvě možnosti jsou ve směru os, třetí je normovaným součtem prvních dvou a poslední je náhodná.
* U hliníku je velikost toku ve všech případech stejná (237) a vždy rovnoběžná se spádem teploty.
* U dřeva je velikost toku tepla v každém ze tří případů jiná a pouze ve dvou případech ze tří je tok rovnoběžný se spádem teploty. V posledním případě teplo neteče směrem, kterým klesá teplota, ale tok se odklání vlivem rozdílných vlastností dřeva v rúzných směrech.

In [14]:
for polozka in ["velikost spádu teploty","velikost toku", "test rovnoběžnosti"]:
    display(df.loc[(slice(None), polozka), :])

,,Hliník,Dřevo
0,velikost spádu teploty,1.0,1.0
1,velikost spádu teploty,1.0,1.0
2,velikost spádu teploty,1.0,1.0
3,velikost spádu teploty,1.0,1.0


,,Hliník,Dřevo
0,velikost toku,237.0,0.3
1,velikost toku,237.0,0.18
2,velikost toku,237.0,0.247386
3,velikost toku,237.0,0.200346


,,Hliník,Dřevo
0,test rovnoběžnosti,True,True
1,test rovnoběžnosti,True,True
2,test rovnoběžnosti,True,False
3,test rovnoběžnosti,True,False


In [15]:
for polozka in polozky:
    display(df.loc[(slice(None), polozka), :].style.format(precision=3))

,,Hliník,Dřevo
0,spád teploty,[1 0],[1 0]
1,spád teploty,[0 1],[0 1]
2,spád teploty,[0.7071 0.7071],[0.7071 0.7071]
3,spád teploty,[0.3665 0.9304],[0.3665 0.9304]


,,Hliník,Dřevo
0,test rovnoběžnosti,True,True
1,test rovnoběžnosti,True,True
2,test rovnoběžnosti,True,False
3,test rovnoběžnosti,True,False


,,Hliník,Dřevo
0,tok,[237 0],[0.3 0. ]
1,tok,[ 0 237],[0. 0.18]
2,tok,[167.5843 167.5843],[0.2121 0.1273]
3,tok,[ 86.8696 220.5055],[0.11 0.1675]


,,Hliník,Dřevo
0,velikost spádu teploty,1.000,1.000
1,velikost spádu teploty,1.000,1.000
2,velikost spádu teploty,1.000,1.000
3,velikost spádu teploty,1.000,1.000


,,Hliník,Dřevo
0,velikost toku,237.000,0.300
1,velikost toku,237.000,0.180
2,velikost toku,237.000,0.247
3,velikost toku,237.000,0.200
